# [pytorch C extension](http://pytorch.org/tutorials/advanced/c_extension.html)

**작성자 : 이상헌 ** <br>

pytorch는 그 사용자 수가 tensorflow에 비하여 절대적으로 작기 때문에 다양한 model, loss가 구현이 되어있지 않다. <br>

또한 torch로 구현하는 것이 속도에 큰 장점이 있기에, torch로 작성된 코드를 pytorch로 extension하는 방법이 존재한다. <br>
(이를 wrapper라 부른다.)

따라서 본 wrapper를 소개하고, 튜토리얼을 통해 그 사용법을 숙달한다. <br>

### stack : <br>
* ubuntu 16.04 <br>
* torch 7 <br>

### 0. install torch

본 [torch](http://torch.ch/docs/getting-started.html#_) 링크를 따라 torch를 설치한다.. <br>

### 1. Pytorch C FFI 예제 시작.

본 예제는 pytorch 공식 github에서 제공하는 예제를 따라, torch로 짜여진 C code를 pytorch로 확장하는 방식을 다룬다. <br>

본 예제를 통하여 extension하는 방식을 알아간다. <br>

### 1-1 installation

가장 본 코드를 다운받는다. <br>
```bash
git clone https://github.com/pytorch/extension-ffi
```
본 repository는 두 개의 example을 가지고 있다. <br>

우리는 ./script 폴더의 예제로 실험을 진행한다. <br>

script 파일은 다음과 같이 이루어져 있다. <br>
```bashscript
├── build.py
├── functions
│   ├── add.py
│   └── __init__.py
├── modules
│   ├── add.py
│   └── __init__.py
├── README.md
├── src
│   ├── my_lib.c
│   ├── my_lib_cuda.c
│   ├── my_lib_cuda.h
│   └── my_lib.h
└── test.py
```

본 프로젝트에서 build.py는 torch로 작성된 모듈(my_lib.c)을 컴파일 해주는 코드이고, <br>
test.py 는 pytorch로 본 모듈을 사용해보는 예시이다. <br>

In [11]:
"""
build.py 의 코드.
FFI = foreign function interface. 다른 language에서 작성된 function 등을 본 language에서 사용하는 바식. 
C FFI (C 언어를 본 pytorch에서 사용할 수 있도록하는 바인딩) 를 pytorch는 제공한다.
torch.utils.ffi 이다.
"""
import os
import torch
from torch.utils.ffi import create_extension


# 본 프로젝트 폴더의 위치를 가져온다.
__file__ = '/home/mappiness/Desktop/deep_learning/Deep_learning/RNN/deep_speech/implementation/extension-ffi/script/'

this_file = os.path.dirname(__file__)

sources = ['src/my_lib.c']
headers = ['src/my_lib.h']
defines = []
with_cuda = False

if torch.cuda.is_available():
    print('Including CUDA code.')
    sources += ['src/my_lib_cuda.c']
    headers += ['src/my_lib_cuda.h']
    defines += [('WITH_CUDA', None)]
    with_cuda = True

ffi = create_extension(
    '_ext.my_lib',
    headers=headers,
    sources=sources,
    define_macros=defines,
    relative_to=__file__,
    with_cuda=with_cuda
)

# FFI는 build의 단계에서 컴파일러가 코드를 해석할 수 있도록 만들어준다. 따라서 함수 이름이 build인 것이다.
ffi.build()

generating /tmp/tmpb8l3o36b/_my_lib.c
running build_ext
building '_my_lib' extension
creating home
creating home/mappiness
creating home/mappiness/Desktop
creating home/mappiness/Desktop/deep_learning
creating home/mappiness/Desktop/deep_learning/Deep_learning
creating home/mappiness/Desktop/deep_learning/Deep_learning/RNN
creating home/mappiness/Desktop/deep_learning/Deep_learning/RNN/deep_speech
creating home/mappiness/Desktop/deep_learning/Deep_learning/RNN/deep_speech/implementation
creating home/mappiness/Desktop/deep_learning/Deep_learning/RNN/deep_speech/implementation/extension-ffi
creating home/mappiness/Desktop/deep_learning/Deep_learning/RNN/deep_speech/implementation/extension-ffi/script
creating home/mappiness/Desktop/deep_learning/Deep_learning/RNN/deep_speech/implementation/extension-ffi/script/src
gcc -pthread -Wsign-compare -DNDEBUG -g -fwrapv -O3 -Wall -Wstrict-prototypes -fPIC -I/home/mappiness/anaconda3/envs/torch/lib/python3.6/site-packages/torch/utils/ffi/../../li

본 FFI를 build하면 다음과 같은 파일들이 생성된다. <br>
```bash
./_ext/
├── __init__.py
└── my_lib
    ├── __init__.py
    └── _my_lib.so
```

이제 본 my_lib을 import하여 사용하면 되는 것이다.

*my_lib.c 코드. 본 코드는 두 node를 더하는 함수의 forward와 backward를 구현해 놓은 함수이다.*

```C
#include <TH/TH.h>

int my_lib_add_forward(THFloatTensor *input1, THFloatTensor *input2,
		       THFloatTensor *output)
{
  if (!THFloatTensor_isSameSizeAs(input1, input2))
    return 0;
  THFloatTensor_resizeAs(output, input1);
  THFloatTensor_cadd(output, input1, 1.0, input2);
  return 1;
}

int my_lib_add_backward(THFloatTensor *grad_output, THFloatTensor *grad_input)
{
  THFloatTensor_resizeAs(grad_input, grad_output);
  THFloatTensor_fill(grad_input, 1);
  return 1;
}
```

1. 이제 단순히 torch.autograd.Function에서 본 함수를 wrapping 해주고, <br>
2. 그 wrapping된 function을 torch.nn.Module을 통해 wrapping 해주면 사용할 수 있다. <br>

여기서 핵심은 function class의 경우에는 forward와 backward가 있지만, <br>
module function은 forward만 정의해주면 된다는 것!

### functions/add.py

``` python
import torch
from torch.autograd import Function
from _ext import my_lib


class MyAddFunction(Function):
    def forward(self, input1, input2):
        output = input1.new()
        if not input1.is_cuda:
            my_lib.my_lib_add_forward(input1, input2, output)
        else:
            my_lib.my_lib_add_forward_cuda(input1, input2, output)
        return output

    def backward(self, grad_output):
        grad_input = grad_output.new()
        if not grad_output.is_cuda:
            my_lib.my_lib_add_backward(grad_output, grad_input)
        else:
            my_lib.my_lib_add_backward_cuda(grad_output, grad_input)
        return grad_input
```

### modules/add.py

```python
from torch.nn.modules.module import Module
from functions.add import MyAddFunction

class MyAddModule(Module):
    def forward(self, input1, input2):
        return MyAddFunction()(input1, input2) # 이 형태가 매우 불안정해.
```
[참고](https://github.com/pytorch/pytorch/issues/821)

### cf) MyAddFunction()(input1, input2) 이란?

function을 보다시피 autograd, 즉 gradient를 저장해놔야 한다. <br>
gradient의 경우 static 변수로 선언이 되어있는데, <br>
이렇게 선언이 되어있으므로 forward의 경우엔 static function으로 callable 하다. <br>
그 function이 단순히 Myfunction()(\*\*kargs) 로 구현이 되어있는 것이다. <br>
반드시 autograd.function은 인스턴스로 만들지 않고 콜링을 해야하며, <br>
forward는 초기화와 함께 인자를 넘기는 방식으로만 사용한다.